# Von Inferenzen und Differenzen
## Ein Vergleich von Topic-Modeling-Engines auf Grundlage historischer Korpora
<i>von Tobias Hodel, Dennis Möbus und Ina Serif</i>

## Einführung

Sie befinden sich nun in einer Programmierkonsole, die auf einer virtuellen Maschine, also nicht auf Ihrem Rechner ausgeführt wird. In den folgenden "Zellen" wechseln sich Textbausteine und Programmierzellen ab, in denen Sie Programmcode ausführen können. Alles ist soweit vorbereitet, dass Sie nur in der oberen linken Ecke einer Zelle zwischen den eckigen Klammern -> [  ] das Play-Symbol klicken müssen, das erscheint, wenn Sie mit der Maus über die Klammern fahren. Die Zellen müssen strikt in der Reihenfolge von oben nach unten ausgeführt werden, da spätere Zellen auf die Variablen voriger aufbauen. In einigen Zellen finden Sie Parameter, die Sie einstellen können - probieren Sie ruhig alles aus und vergleichen Sie die Ergebnisse.


## Installation von Java und Mallet

In [ ]:
! java -version
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip -q mallet-2.0.8.zip
!mallet-2.0.8/bin/mallet
mallet_path = 'mallet-2.0.8/bin/mallet' 


## Installation und Import benötigter Pakete

In [ ]:
!pip install -i https://test.pypi.org/simple/ topic-modeling-moebusd==0.27
!pip install pandas
!pip install gensim==3.8.3
!pip install plotly
import os
import pandas as pd
import plotly.express as px
from topic_modeling import remove_stopwords_by_threshold
from topic_modeling import remove_stopwords_by_list
from topic_modeling import lemmatization

## Import der Daten

Bitte gewünschte Zelle ausführen!

<i>Mittelalterliche Handschriften laden</i>

In [ ]:
chunksize = 500 # Länge der Chunks in Wörtern, default ist 500, mögliche Werte: 250, 500, 1000, 2500, 5000
                # möglich ist auch die Angabe "fulltext" (mit Anführungszeichen!)

name_dataset = 'twinger_chronik_chunked_'+str(chunksize)+'_Wörter' 
dataframeimport = pd.read_pickle(name_dataset) 
raw_data = dataframeimport.values.tolist() # Umwandlung des Dataframes in eine Liste
data = [[word for word in line[1].split()] for line in raw_data] # Tokenisierung
print(data[0][:9])

<i>Schweizer Ratsprotokolle des 19. Jahrhunderts laden</i>

In [ ]:
files_resolutions = 'https://raw.githubusercontent.com/DHBern/TopicModeling/main/Data/dataframes/data_stazh_tkr_df'
dataframeimport = pd.read_pickle(files_resolutions)
name_dataset = 'resolutions'
raw_data = dataframeimport.values.tolist() # Umwandlung des Dataframes in eine Liste
data = [[word for word in line[1].split()] for line in raw_data] # Tokenisierung
print(data[0][:9])

<i>Oral-History-Interviews des 20. Jahrhunderts laden</i>

In [ ]:
chunksize = 25 # Länge der Chunks in Sätzen, default ist 25, mögliche Werte: 1, 5, 10, 25, 50

name_dataset = 'all_transcripts_df_sentences_clean_normalized_'+str(chunksize)+'sentence(s)_NEW'
dataframeimport = pd.read_pickle(name_dataset)
raw_data = dataframeimport.values.tolist() # Umwandlung des Dataframes in eine Liste
data = [[word for word in line[1].split()] for line in raw_data] # Tokenisierung
print(data[0][:10])

## Preprocessing

### Entfernen von Stopwords durch Stoplist

In [ ]:
if 'twinger' in name_dataset:
    stoplist = open('gmh_stopwords_edited.txt', encoding='UTF-8', mode='r').read().split()
else:
    stoplist = open('german_stopwords_full_BE_MOD Topics_UTF-8.txt', encoding='UTF-8', mode='r').read().split()

data_nostops = remove_stopwords_by_list(data, stoplist)
print(data_nostops[:9])

### Entfernen von Stopwords durch Threshold

In [ ]:
threshold = 0.001 # Alle Wörter oberhalb dieses prozentualen Anteils am Text werden heraus gefiltert

data_nostops = remove_stopwords_by_threshold(data, threshold)
print(data_nostops[:9])

### Lemmatisierung (nur für Oral-History-Interviews)

In [ ]:
# Zusätzliche Pakete installieren
!pip install --upgrade spacy
!pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_lg-3.2.0/de_core_news_lg-3.2.0.tar.gz
import spacy

In [ ]:
data_lemmatized = lemmatization(data_nostops, 3)    # als erstes Argument kann auch "data" übergeben werden, wenn nur lemmatisiert werden soll, 
print(data_lemmatized[:9])                          # ohne vorher Stopwords zu entfernen
                                                    # als zusätzliches Argument kann allowed_postags=['NN', 'PN', 'ADJ', 'ADV', 'VB'] angegeben werden, 
                                                    # wobei die Postags in der Liste auch auf weniger reduziert werden können

## Topic Training

### Gensim (= Variational Bayes)

In [ ]:
from topic_modeling import topic_training_gensim
lda_model_gensim, doc_tops_gensim, topwords_gensim = topic_training_gensim(data_nostops, name_dataset, 'moebusd', 50, passes_gensim=5, iterations_gensim=10)

### Mallet (= Gibbs-Sampling)

In [ ]:
from topic_modeling import topic_training_mallet
lda_model_mallet, doc_tops_mallet, topwords_mallet = topic_training_mallet(data, name_dataset, 'moebusd', 50, mallet_path, optimize_interval_mallet=50, iterations_mallet=500)

## Topic-Listen ausgeben

### Gensim

In [ ]:
from topic_modeling import print_topics_gensim
print_topics_gensim(topwords_gensim, 15, name_dataset)

### Mallet

In [ ]:
from topic_modeling import print_topics_mallet
print_topics_mallet(topwords_mallet, 15, name_dataset)

## Evaluation und Visualisierung

In [ ]:
from topic_modeling import doc_top_heatmap

doc_top_heatmap(doc_tops_mallet, topwords_mallet, raw_data, split_index=1) # Welche Bestandteile der Signatur für Y-Achse?